In [12]:
sql = 'SELECT address,goto,domain,active from alias;'
print(common._run_sql_on_maildb(domain.api_url, domain.api_token, domain.namespace, domain.label, sql))

mysql -u postfix -p$MYSQL_PASSWORD -D postfix -e 'SELECT address,goto,domain,active from alias;'
address	goto	domain	active
@exrp.pl	catchall@exrp.pl	exrp.pl	1
abuse@exrp.pl	abuse@	exrp.pl	1
admin@exrp.pl	a_suh@exrp.pl	exrp.pl	1
a_suh@exrp.pl	a_suh@exrp.pl	exrp.pl	1
catchall@exrp.pl	catchall@exrp.pl	exrp.pl	1
g_user@exrp.pl	g_user@exrp.pl	exrp.pl	1
hostmaster@exrp.pl	hostmaster@	exrp.pl	1
jumshud@exrp.pl	nurlan@exrp.pl,g_user@exrp.pl	exrp.pl	0
normal@exrp.pl	normal@exrp.pl	exrp.pl	1
nurlan.farajov@exrp.pl	nurlan.farajov@exrp.pl	exrp.pl	1
nurlan@exrp.pl	nurlan.farajov@exrp.pl,nurlan@erpgo.az	exrp.pl	1
nurlantest@exrp.pl	nurlantest@exrp.pl	exrp.pl	1
postfixadmin@exrp.pl	admin@exrp.pl	exrp.pl	1
postmaster@exrp.pl	postmaster@	exrp.pl	1
test222@exrp.pl	nurlan@exrp.pl	exrp.pl	1
test@exrp.pl	nurlan@exrp.pl	exrp.pl	1
webmaster@exrp.pl	webmaster@	exrp.pl	1



In [36]:
from models import common

In [37]:
domain = env['mail_reset.domain'].search([])

In [38]:
def _get_record_data(output, fields):
    outlist = output.splitlines()[1:]
    record_data = []
    for l in outlist:
        l = l.split('\t')
        data = dict(zip(fields, l))
        data['address'] = data['address'].split('@')[0]
        data['active'] = int(data['active'])
        record_data.append(data)
    return record_data


In [39]:
def _sync_aliases(domain):
    sql = 'SELECT address,goto,domain,active from alias;'
    output = common._run_sql_on_maildb(domain.api_url, domain.api_token, domain.namespace, domain.label, sql)
    fields=['address','goto','domain','active']
    
    return _get_record_data(output, fields)

In [42]:
records = _sync_aliases(domain)

mysql -u postfix -p$MYSQL_PASSWORD -D postfix -e 'SELECT address,goto,domain,active from alias;'


In [43]:
records

[{'address': '', 'goto': 'catchall@exrp.pl', 'domain': 'exrp.pl', 'active': 1},
 {'address': 'abuse', 'goto': 'abuse@', 'domain': 'exrp.pl', 'active': 1},
 {'address': 'admin',
  'goto': 'a_suh@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'a_suh',
  'goto': 'a_suh@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'catchall',
  'goto': 'catchall@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'g_user',
  'goto': 'g_user@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'hostmaster',
  'goto': 'hostmaster@',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'jumshud',
  'goto': 'nurlan@exrp.pl,g_user@exrp.pl',
  'domain': 'exrp.pl',
  'active': 0},
 {'address': 'normal',
  'goto': 'normal@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'nurlan.farajov',
  'goto': 'nurlan.farajov@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'nurlan',
  'goto': 'nurlan.farajov@exrp.pl,nurlan@erpgo.az',
  'domain': 'exrp

In [34]:
records = _sync_aliases(domain)
all_aliases = env['mail_reset.aliases'].with_context(active_test=False)
for alias in all_aliases:
    alias.unlink()
for data in records:
    data['name'] = f"{data['address']}@{data['domain']}"
    data['domain'] = env['mail_reset.domain'].search([('name','=', data['domain'])]).id
    env['mail_reset.aliases'].create(data)


mysql -u postfix -p$MYSQL_PASSWORD -D postfix -e 'SELECT address,goto,domain,active from alias;'


ValidationError: ('Address is invalid!', None)

In [70]:
data['active'] = int(data['active'])

In [35]:
records

[{'address': '',
  'goto': 'catchall@exrp.pl',
  'domain': 1,
  'active': 1,
  'name': '@exrp.pl'},
 {'address': 'abuse', 'goto': 'abuse@', 'domain': 'exrp.pl', 'active': 1},
 {'address': 'admin',
  'goto': 'a_suh@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'a_suh',
  'goto': 'a_suh@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'catchall',
  'goto': 'catchall@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'g_user',
  'goto': 'g_user@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'hostmaster',
  'goto': 'hostmaster@',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'jumshud',
  'goto': 'nurlan@exrp.pl,g_user@exrp.pl',
  'domain': 'exrp.pl',
  'active': 0},
 {'address': 'normal',
  'goto': 'normal@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'nurlan.farajov',
  'goto': 'nurlan.farajov@exrp.pl',
  'domain': 'exrp.pl',
  'active': 1},
 {'address': 'nurlan',
  'goto': 'nurlan.farajov@exrp.pl,nurlan@erpgo.az

In [15]:
address = 'nurlan@exrp.pl'
address.split('@')[0]

'nurlan'